### Import package

In [1]:
import numpy as np
from pypuf.simulation import *
import pypuf.metrics as pm
from pypuf.io import random_inputs
import pypuf.io, pypuf.simulation
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import metrics, activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from mmoe import MMoE
import datetime
import psutil
from tensorflow.python.profiler import profiler_client
from preprocessing import *

process = psutil.Process()
memory_info_start = process.memory_info()

# 打印内存使用情况
print(f"Memory: {memory_info_start.rss / (1024 ** 2):.2f} MB")


Memory: 264.51 MB


### Important callbacks and the MoPE model

In [2]:
class SingleDynamicThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, custom_layer, **kwargs):
        super(SingleDynamicThresholdCallback, self).__init__(**kwargs)
        self.custom_layer = custom_layer

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current_accuracy = logs.get('accuracy')
        if current_accuracy>0.95:
            self.model.stop_training = True
            print("Early Stop!")
        elif current_accuracy>0.90:
            K.set_value(self.custom_layer.threshold,0.001)
        elif current_accuracy>0.8:
            K.set_value(self.custom_layer.threshold,0.0005)
        elif current_accuracy>0.7:
            K.set_value(self.custom_layer.threshold,0.0001)


                
class MemoryUsageCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % 100 == 0:
            process = psutil.Process()
            memory_info = process.memory_info()
            print(f"Epoch {epoch + 1} - Memory Cost: {memory_info.rss / (1024 ** 2):.2f} MB")
memory_callback = MemoryUsageCallback()

def Model_Multiple_PUFs(units_mmoe,gate_activation,N_train,train_c,test_c,train_r_groups,test_r_groups,PUF_list,units_tower=2):    
    num_features = 64
    input_layer = Input(shape=(num_features,))
    experts_list = []
    activation = 'relu'
    kernel_init = 'glorot_uniform'
    drop_out_rate = 0.2
    num_neu = 2**5

    num_expert = 7
    experts_list = [Expert_customize([num_neu,num_neu],activation=activation,kernel_init=kernel_init,drop_out_rate=drop_out_rate)
                    for _ in range(num_expert)]
    tower_units = [num_neu for _ in range(PUF_list.n_pufs)]


    # Set up MMoE layer
    mmoe_layers = MMoE(
        units=units_mmoe,
        use_gate_bias=True,
        experts=experts_list,
        num_tasks=PUF_list.n_pufs,
        expert_activation=activation,
        gate_activation=gate_activation,
        dropout_rate=0.1,
    )(input_layer)


    output_layers = []
    output_info = ['r'+str(i) for i in range(1,PUF_list.n_pufs+1)]
    train_losses = {output: [] for output in output_info}
    train_accuracies = {output: [] for output in output_info}
    val_losses = {output: [] for output in output_info}
    val_accuracies = {output: [] for output in output_info}

    # Build tower layer from MMoE layer
    for index, task_layer in enumerate(mmoe_layers):
        tower_layer = Dense(
            units=tower_units[index],
            activation=activation)(task_layer)
        output_layer = Dense(
            units=1,
            name=output_info[index],
            activation='sigmoid',
            kernel_initializer=kernel_init)(tower_layer)
        output_layers.append(output_layer)

    # Compile model
    model = Model(inputs=[input_layer], outputs=output_layers)
    optimizer = 'adam'
    model.compile(
        loss='binary_crossentropy',
        optimizer = optimizer,
        metrics=['accuracy']
    )

    SingleDynamic_Threshold = SingleDynamicThresholdCallback(gate_activation[0])
    # Print out model architecture summary
    model.summary()
    batch_size = min(N_train//40,20000)

    history = model.fit(
        x=train_c,
        y=train_r_groups[0],
        validation_data=(test_c, test_r_groups[0]),
        batch_size=batch_size,
        epochs=2000
        ,callbacks = [memory_callback,SingleDynamic_Threshold]

    )
    return model, history,output_info,train_losses,train_accuracies,val_losses,val_accuracies


### Data Preparation
Only need to choose which PUF and how many CRPs, do not need to change the model.

In [3]:
import random
PUF_list = PUFs(stages=64)
PUF_list.seed = random.randint(1,1000)
print("seed=",PUF_list.seed)
CRP_seed = random.randint(1,100)
print("CRP seed=",CRP_seed)

N_train = 3000000
# Choose one PUF to be model
PUF_list.add_XOR_PUF(k=7,num=1)


## Below can be uncommented as your will
# PUF_list.add_XOR_PUF(k=2,num=1)
# PUF_list.add_XOR_PUF(k=3,num=1)
# PUF_list.add_XOR_PUF(k=4,num=1)
# PUF_list.add_XOR_PUF(k=5,num=1)
# PUF_list.add_XOR_PUF(k=6,num=1)
# PUF_list.add_FF_PUF([(32,50)],1)
# PUF_list.add_herero_XORFF_PUFs(k=2,ff=[[(20,50),(19,52)],[(21,54),(22,55)],[(30,60),(31,62)]],num=1)
# PUF_list.add_herero_XORFF_PUFs(k=3,ff=[[(20,50)],[(21,54)],[(30,60)]],num=1)
# PUF_list.add_interpose_PUFs(1,5,1)
# PUF_list.add_interpose_PUFs(2,2,1)
# PUF_list.add_interpose_PUFs(3,3,1)
# PUF_list.add_XORFF_PUF(2,[(20,50)],1)
# PUF_list.add_XORFF_PUF(4,[(20,50),(21,54)],1)
# PUF_list.add_XORFF_PUF(3,[(20,50),(21,54)],1)
# PUF_list.add_XORFF_PUF(2,[(21,54)],1)
# PUF_list.add_XORFF_PUF(2,[(21,54),(10,40)],1)
# PUF_list.add_XORFF_PUF(1,[(20,50),(10,40),(30,60)],1)
# PUF_list.add_XOR_PUF(1,num=1)

[c, responses] = PUF_list.generate_crps(CRP_seed,N_train)
c = get_parity_vectors2(c)
# c = np.concatenate([c,get_parity_vectors2(c)],axis=1)
responses = np.array(responses)
# print(responses.shape)
train_c,test_c,train_r,test_r = train_test_split(c,responses.T,test_size=0.2, random_state=42)
train_r_groups = [train_r[:,i].reshape(-1,1) for i in range(PUF_list.n_pufs)]
test_r_groups = [test_r[:,i].reshape(-1,1) for i in range(PUF_list.n_pufs)]


seed= 922
CRP seed= 100


We also provide a demo of using real data. The only thing you need to do is changing the datasets.

In [ ]:
# path = 'data/4To9_XPUF_Silicon_CRPs/7XOR_64bit_LUT_2239B_attacking_5M.txt'
# path = 'data/7puf_output_2000000.txt'
# data = pd.read_csv(path,sep=';',header=None)
# challenges = np.asarray(data[0].apply(lambda x: bin(int(x, 16))[2:].zfill(64)))
# responses = []
# for k in range(7):
#     responses.append(np.asarray(data[1])//(10**k)%10)
# response_XOR2 = responses[1] ^ responses[2]
# challenges = np.array([[int(bit) for bit in ch] for ch in challenges])

In [4]:

import itertools

units_mmoe = 32
units_tower = 16
model = None
import pandas as pd
######################################################################
gate_activation = [CustomSoftmaxThre(threshold=0.00001) for _ in range(PUF_list.n_pufs)]
# Below is the normal softmax activation.
# gate_activation = [activations.get('softmax')]


model,history, output_info, train_losses, train_accuracies, val_losses, val_accuracies = Model_Multiple_PUFs(
    units_mmoe=units_mmoe,
    gate_activation=gate_activation,
    N_train=N_train,
    train_c=train_c,
    test_c=test_c,
    train_r_groups=train_r_groups,
    test_r_groups=test_r_groups,
    PUF_list=PUF_list,
    units_tower=units_tower)

memory_info_end = process.memory_info() 

# print memory cost
print(f"Memory cost: {(memory_info_end.rss- memory_info_start.rss) / (1024 ** 3):.2f} GiB")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
m_mo_e (MMoE)                [(None, 32)]              22408     
_________________________________________________________________
dense_14 (Dense)             (None, 32)                1056      
_________________________________________________________________
r1 (Dense)                   (None, 1)                 33        
Total params: 23,497
Trainable params: 23,497
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2000
120/120 [==============================] - 3s 16ms/step - loss: 0.6988 - accuracy: 0.5003 - val_loss: 0.6937 - val_accuracy: 0.5005
Epoch 2/2000
120/120 [==============================] - 1s 10ms/step - loss: 0.6935 - accuracy: 0.5006 - val_l

Here we show one example of modelling 7-XOR APUF. For this case, the training is very fast, we also admit that it may increase for different random seeds.